In [3]:
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.storage import LocalFileStore
from langchain.schema import Document
import os

In [6]:
llm = ChatOpenAI(temperature=0.1, streaming=True)

# LLM을 활용한 문서 요약 함수
def summarize_documents(docs, llm):
    summarized_docs = []
    for doc in docs:
        # 요약 요청을 LLM으로 보내기
        summary = llm.predict(f"이 내용 요약해줘: {doc.page_content}")
        
        # Document 객체에 담아 summarized_docs에 추가 
        summarized_docs.append(Document(page_content=summary))
    return summarized_docs

In [ ]:
def embed_file(file, summarize=False):
    file_content = file.read()  # 파일의 내용을 읽어오기
    file_path = f"./.cache/files/{file.name}"  # 저장될 파일의 경로
    with open(file_path, "wb") as f:
        f.write(file_content)  # 선택한 파일의 내용을 .cache/files 디렉토리로 옮김

    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
    )

    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".[:5] | .[].content",  # 5개의 기사만 가져오기
        text_content=True,
    )

    data = loader.load()
    
    # 요약하기 버튼을 선택한 경우
    if summarize:
        # 요약한 문서의 임베딩 결과를 저장할 디렉토리 만들기
        cache_dir_path = f"./.cache/summarized_embeddings/{file.name}"
        os.makedirs(cache_dir_path, exist_ok=True) # 디렉토리가 없으면 만들기
        cache_dir = LocalFileStore(cache_dir_path)
        
        docs = splitter.split_documents(data)  # splitter에 맞게 문서 분할
        
        # 문서 요약
        docs = summarize_documents(docs, llm)
        
        embeddings = OpenAIEmbeddings()

        # 중복 요청 시 캐시된 결과를 반환
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            embeddings, cache_dir
        )
        
    # 요약하기 버튼을 선택하지 않은 경우
    else:
        # 문서의 임베딩 결과를 저장할 디렉토리 만들기
        cache_dir_path = f"./.cache/embeddings/{file.name}"
        os.makedirs(cache_dir_path, exist_ok=True) # 디렉토리가 없으면 만들기
        cache_dir = LocalFileStore(cache_dir_path)
        
        docs = splitter.split_documents(data)  # splitter에 맞게 문서 분할

        embeddings = OpenAIEmbeddings()

        # 중복 요청 시 캐시된 결과를 반환
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            embeddings, cache_dir
        )

    # FAISS 라이브러리로 캐시에서 임베딩 벡터 검색
    vectorstore = FAISS.from_documents(docs, cached_embeddings)

    # docs를 불러오는 역할
    retriever = vectorstore.as_retriever()

    return retriever